In [85]:
import os
import json
from tqdm import tqdm

In [86]:
def file_open(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

In [87]:
base_path = '../data/document/역도'

name_list = ['역도의 이해', '역도의 스포츠 과학적 원리', '역도경기 기술의 구조와 훈련법', '역도체력의 구조와 훈련법', '역도 훈련프로그램 구성 및 지도안']
path_list = []

suffix = '_dataset.json'
for name in name_list:
    path = os.path.join(base_path, name + suffix)
    path_list.append(path)

dataset_list = []
for path in path_list:
    print(path)
    file = file_open(path)
    dataset_list.append(file)

../data/document/역도\역도의 이해_dataset.json
../data/document/역도\역도의 스포츠 과학적 원리_dataset.json
../data/document/역도\역도경기 기술의 구조와 훈련법_dataset.json
../data/document/역도\역도체력의 구조와 훈련법_dataset.json
../data/document/역도\역도 훈련프로그램 구성 및 지도안_dataset.json


# 문서 전처리
## 검색 전용 데이터 생성
1. 불필요 정보 제거
   - figure, chart, table의 hypotheticalQuestions 제거(?)
   - filepaths 제거
   - category 제거 
2. 페이지 수정
3. 개별 문서, 병합 문서 생성

### 검색 전용 Document 구조
- id: int
- metadata: Dict
  - filename: str
  - page: List[int]
- page_content: str
- type: Document

In [88]:
def make_dataset_for_search(dataset, before_page, before_id):
    new_dataset = []
    for i, doc in enumerate(tqdm(dataset['documents'])):
        new_document = {'id': before_id + i + 1,
                        'metadata': {},
                        'page_content': ''}

        metadata = doc['meta']
        metadata['filename'] = metadata['filepath'].split('/')[-1]
        del metadata['filepath']

        page_list = metadata['pages']
        page_list = [page + before_page for page in page_list]
        del metadata['pages']

        new_document['metadata'] = metadata
        new_document['metadata']['page'] = page_list

        page_content = ''
        for element in doc['content']:
            if element['category'] == 'table':
                json = element['information']['table_json'] # 문자열
                page_content += json + '\n'

            if element['category'] in ['figure', 'chart', 'table']:
                if 'caption' in element['information']:
                    caption = element['information']['caption']
                    page_content += caption + '\n'
                detail = element['information']['detail']
                hypotheticalQuestions = element['information']['hypotheticalQuestions']
                hypotheticalQuestions = ' '.join([question for question in hypotheticalQuestions])
                page_content += detail + '\n'
                page_content += hypotheticalQuestions + '\n'
            
            if element['category'] == 'paragraph':
                page_content += element['information'] + '\n'
        
        new_document['page_content'] = page_content
        new_dataset.append(new_document)
    
    before_page = page_list[-1]
    print(before_page)
    before_id = before_id + len(dataset['documents'])
    return new_dataset, before_page, before_id

In [89]:
new_dataset_list = []
before_page = 0
before_id = 0

for i, dataset in enumerate(dataset_list):
    new_dataset, before_page, before_id = make_dataset_for_search(dataset, before_page, before_id)
    new_dataset_list.append(new_dataset)

100%|██████████| 32/32 [00:00<00:00, 63370.03it/s]


23


100%|██████████| 81/81 [00:00<00:00, 81511.19it/s]


100


100%|██████████| 104/104 [00:00<00:00, 82349.94it/s]


180


100%|██████████| 23/23 [00:00<?, ?it/s]


199


100%|██████████| 36/36 [00:00<?, ?it/s]

216


In [94]:
merged_dataset = []
for i, dataset in enumerate(new_dataset_list):
    merged_dataset.extend(dataset)

In [95]:
save_dataset = {'new_dataset': new_dataset_list, 'merged_dataset': merged_dataset}
# with open('../data/document/역도/save_dataset.json', 'w', encoding='utf-8') as f:
#     json.dump(save_dataset, f, ensure_ascii=False, indent=4)

# RAGAS 실험
### 평가 항목
1. Generation 
   1. Faithfulness
      * 주어진 문맥에 대한 생성된 답변의 사실적 일관성 측정
      * 답변과 검색된 문맥(retrieved context)를 기준으로 계산
      * (0, 1) 범위 스케일이며, 값이 높을수록 좋음
      * 생성된 답변이 신뢰할 수 있다고(faithful) 간구되려면 답변에서 제시된 모든 주장이 주어진 문맥(given context)에서 추론될 수 있어야 함
      * 생성된 답변에서 주장의 집합(claims)를 식별 -> 각 주장마다 주어진 맥락 기반 여부 확인
      * 점수: context 기반의 답변 내 주장 수 / 전체 주장 수
      * 예시
        * 아이슈타인의 출생일자와 출생지는 어디인가?
          * 답변 1: 아이슈타인은 독엘에서 1879/3/14에 태어났습니다.
            * 
          * 답변 2: 아이슈타인은 독엘에서 1879/4/14에 태어났습니다.
            * 
          * context: 
* 
   1. 
1. Retriever
   1. 
## 1. 